<p align="center">
  <img align="right" src="https://nilearn.github.io/dev-days-2020/assets/images/nilearn-logo.png" alt="image" width="250" />
</p>

<h1 align="center">MultiRat Sensory-Evoked analysis code</h1>

<p align="center">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<i>Marie E Galteau, 2023</i></p>

---

## RABIES preprocessing

The present jupyter notebook contains the code to reproduce the preprocessing steps using RABIES pipeline. 

Follow the code step-wise. 


In [1]:
# -- Load module for singularity -- 

!module load singularity
!module load freesurfer
!module load fsl

import os 
import subprocess
import pandas as pd
from enum import Enum

from IPython.display import display, Markdown, Latex

# -- Init Variables -- 
metadata_path='/home/traaffneu/margal/code/multirat_se/script/table/metadata_rabies.tsv'        # meta-data table

df = pd.read_csv(metadata_path, sep='\t')
df = df.loc[(df['exclude'] != 'yes')]

#### Define RABIES parameters in a function

In [2]:
#--- Function to submit job to HPC --- 

def qsub_prepro_rabies(subj_num, template_dir, BIDS_input, preprocess_outputs, TR, template, mask, wm, csf, atlas, correction_arg):
    
    if not os.path.exists(preprocess_outputs):os.makedirs(preprocess_outputs)                    # Create directory
    singularity_path='/opt/singularity/3.10.3/bin/singularity'
    
    # -- Create the full command string --    
    rabies_cmd = f"{singularity_path} run " \
                f"-B {template_dir}:/template " \
                f"-B {BIDS_input}:/BIDS_input:ro " \
                f"-B {preprocess_outputs}:/preprocess_outputs " \
                "/opt/rabies/0.4.8/rabies.sif -p MultiProc preprocess /BIDS_input /preprocess_outputs " \
                    f"--TR {TR} " \
                    f"--anat_template {template} " \
                    f"--brain_mask {mask} " \
                    f"--WM_mask {wm} " \
                    f"--CSF_mask {csf} " \
                    f"--vascular_mask {csf} " \
                    f"--labels {atlas} " \
                    "--commonspace_reg='masking'='false','brain_extraction'='false','template_registration'='SyN','fast_commonspace'='true' " \
                    "--commonspace_resampling '0.3x0.3x0.3' " \
                    f"{correction_arg} " \

    cmd = f'echo "Job running: $(echo \'{rabies_cmd}\'| qsub -N MRat:{subj_num} -l nodes=1:ppn=1,mem=64gb,walltime=48:00:00)"'
    subprocess.call(cmd, shell=True)


#### Preprocess each scan

- Define scans info specified in the metadata file: subject number, TR, session, additional preprocessing parameters. 
- We define path to the rat SIGMA templates as well as input and output path. 
- We register in the metadata wether scans must be preprocessed with default (#0) or corrective (#2) parameters.


In [4]:
# --- Preprocess all datasets with RABIES --- 

for index in range(0, len(df)):

    # --- Define scans variables ---
    rabies_preprocess = str(df.iloc[index]['rabies_prepro'])         #remove the 2 last characters with [:-2], because input = 0.0 instead of 0
    subj_num=str(df.iloc[index]['rat.sub'])[:-2]                               #remove the 2 last characters with [:-2], if need zero at the begenning .zfill(9) )
    ses_num=str(df.iloc[index]['rat.ses'])[:-2] 
    TR=str(df.iloc[index]['func.TR'])
        
    # --- Template & masks --
    template_dir='/groupshare/traaffneu/preclinimg/templates/SIGMA_Wistar_Rat_Brain_TemplatesAndAtlases_Version1.1'
    template='/template/SIGMA_Rat_Anatomical_Imaging/SIGMA_Rat_Anatomical_InVivo_Template/SIGMA_InVivo_Brain_Template.nii'
    mask='/template/SIGMA_Rat_Anatomical_Imaging/SIGMA_Rat_Anatomical_InVivo_Template/SIGMA_InVivo_Brain_Mask.nii'
    wm='/template/SIGMA_Rat_Anatomical_Imaging/SIGMA_Rat_Anatomical_InVivo_Template/SIGMA_InVivo_WM_bin.nii.gz'
    csf='/template/SIGMA_Rat_Anatomical_Imaging/SIGMA_Rat_Anatomical_InVivo_Template/SIGMA_InVivo_CSF_bin.nii.gz '
    atlas='/template/SIGMA_Rat_Brain_Atlases/SIGMA_Anatomical_Atlas/SIGMA_Anatomical_Brain_Atlas_rs.nii'

    # --- Input and output directories ---
    BIDS_input=f'/project/4180000.19/multirat_stim/rabies/BIDS_input/sub-0{subj_num}'
    preprocess_outputs=f'/project/4180000.19/multirat_stim/scratch/rabies_test/preprocess/sub-0{subj_num}_ses-{ses_num}'
    
    class ProcessingConfig(Enum):
        DefaultParameters='0'
        AlreadyPreprocessed='1'
        CorrectiveParameters='2'
        
    rabies_preprocess_config_value=str(df.iloc[index]['rabies_prepro'])    
    rabies_preprocess_config = ProcessingConfig(rabies_preprocess_config_value)
    
    # --- Print scan infos --- 
    scan_infos=(f'Rat num: <span style="color:#FF5733">0{subj_num}</span>  \n'     
                f'Preprocess config: <span style="color:#FF91A4">{rabies_preprocess_config_value}</span>  \n'
                f'Session n°: <span style="color:#45B39D">{ses_num}</span>  \n'
                f'TR: <span style="color:#0099FF">{TR}</span>  \n')

    
    # -- Run RABIES preprocessing, as a job on the HPC --
    if rabies_preprocess_config == ProcessingConfig.CorrectiveParameters:
        display(Markdown(scan_infos))        
        correction_arg=str(df.iloc[index]['rabies_cor'])
        qsub_prepro_rabies(subj_num, template_dir, BIDS_input, preprocess_outputs, TR, template, mask, wm, csf, atlas, correction_arg)
        print("RABIES: corrective params.")
        print('-----------')  
        
    elif rabies_preprocess_config == ProcessingConfig.DefaultParameters:        
        display(Markdown(scan_infos))
        correction_arg=''
        qsub_prepro_rabies(subj_num, template_dir, BIDS_input, preprocess_outputs, TR, template, mask, wm, csf, atlas, correction_arg)
        print("RABIES: default params.") 
        print('-----------')  

Rat num: <span style="color:#FF5733">0202302</span>  
Preprocess config: <span style="color:#FF91A4">0</span>  
Session n°: <span style="color:#45B39D">1</span>  
TR: <span style="color:#0099FF">1.0</span>  


Job running: 51525804.dccn-l029.dccn.nl
RABIES: default params.
-----------


Rat num: <span style="color:#FF5733">0202303</span>  
Preprocess config: <span style="color:#FF91A4">0</span>  
Session n°: <span style="color:#45B39D">1</span>  
TR: <span style="color:#0099FF">1.0</span>  


Job running: 51525805.dccn-l029.dccn.nl
RABIES: default params.
-----------


Rat num: <span style="color:#FF5733">0202304</span>  
Preprocess config: <span style="color:#FF91A4">0</span>  
Session n°: <span style="color:#45B39D">1</span>  
TR: <span style="color:#0099FF">1.0</span>  


...

Job running: 51525809.dccn-l029.dccn.nl
RABIES: default params.
-----------


Rat num: <span style="color:#FF5733">0202308</span>  
Preprocess config: <span style="color:#FF91A4">0</span>  
Session n°: <span style="color:#45B39D">1</span>  
TR: <span style="color:#0099FF">1.0</span>  


Job running: 51525810.dccn-l029.dccn.nl
RABIES: default params.
-----------


Rat num: <span style="color:#FF5733">0202309</span>  
Preprocess config: <span style="color:#FF91A4">0</span>  
Session n°: <span style="color:#45B39D">1</span>  
TR: <span style="color:#0099FF">1.0</span>  


Job running: 51525811.dccn-l029.dccn.nl
RABIES: default params.
-----------
